In [1]:
import pandas as pd 
from sklearn.preprocessing import StandardScaler
import lightgbm as lgb
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

# Only read up to 4096 bytes, > 4096 has 100% malware rate
MAX_SIZE = 4096
TOTAL_ROWS = 113636
USE_COLS = list(range(2, MAX_SIZE))
# Change this to test
ROWS = 80000

x_train = pd.read_csv("./data/train/train.zip", nrows=ROWS, usecols=USE_COLS, header=None, names = list(range(0, MAX_SIZE)))
# x_train = np.array(x_train)
# x_train = sc.fit_transform(x_train)
y_train = pd.read_csv("./data/train_label.csv", usecols=[1], nrows=ROWS)
# y_train = np.reshape(np.array(y_train), ROWS)

x_train = x_train.fillna(0, downcast='infer')
x_train = x_train.values
y_train = y_train.values

# x_train = np.nan_to_num(x_train)

assert x_train.shape[0] == y_train.shape[0], "Train and label shapes are different"

In [5]:
# # Import all data
# import lightgbm as lgb
# from operator import itemgetter

# # Save model
# bgt = lgb.Booster(model_file="./model-99034.txt")

# feature_indices = bgt.feature_name()
# feature_importance = bgt.feature_importance()

# feature_rank = []

# for i, index in enumerate(feature_indices):
#     importance = feature_importance[i]
#     if importance == 0:
#         continue
    
#     feature_rank.append((index, importance))

# feature_rank.sort(key=itemgetter(1), reverse=True)
    
# mask=np.fromiter(map(lambda x: x[0], feature_rank[0: 900]), dtype=np.int)

In [2]:
# x_train_small = x_train[:, mask]
x_train.shape

(80000, 4094)

In [8]:
x_train_small = x_train
import keras
from keras import Sequential
from keras.layers import *
from keras.models import Model
from keras import optimizers

num_classes = 2
batch_size = 128
epochs = 40
own_embedding_vocab_size=256

input1 = Input(shape=(4094, ))
print(input1._keras_shape)

embedding = Embedding(input_dim=own_embedding_vocab_size, 
                      output_dim=8)(input1)

conv_normal = Conv1D(filters=128, kernel_size=5, strides=2,
                     activation='elu')(embedding)

conv_sigmoid = Conv1D(filters=128, kernel_size=5, strides=2,
                     activation='sigmoid')(embedding)

merged_layer = Multiply()([conv_normal, conv_sigmoid])
bn1 = BatchNormalization()(merged_layer)

relu_layer = Activation('relu')(bn1)

temporal_max = GlobalMaxPooling1D()(relu_layer)

dense = Dense(128, activation='elu')(temporal_max)
bn2 = BatchNormalization()(dense)

output = Dense(1, activation='softmax')(bn2)

model = Model(inputs=input1, outputs=output)

print(model.summary())

adam=optimizers.Adam(lr=0.003)

model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['acc'])

model.fit(x_train_small, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1)

model.save('cnn_model_custom.h5')

(None, 4094)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 4094)         0                                            
__________________________________________________________________________________________________
embedding_6 (Embedding)         (None, 4094, 8)      2048        input_6[0][0]                    
__________________________________________________________________________________________________
conv1d_11 (Conv1D)              (None, 2045, 128)    5248        embedding_6[0][0]                
__________________________________________________________________________________________________
conv1d_12 (Conv1D)              (None, 2045, 128)    5248        embedding_6[0][0]                
________________________________________________________________________________________________

KeyboardInterrupt: 